In [3]:
%pip install boto3

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 129 kB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 517 kB/s  eta 0:00:01
     |████████████████████████████████| 6.5 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 47 kB/s s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import lightgbm as lgb
import matplotlib as mpl

import mlflow
import mlflow.lightgbm
import mlflow.sklearn

In [2]:
learning_rate=0.4
colsample_bytree = 0.7 
subsample = 0.8
# prepare train and test data
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_set = lgb.Dataset(X_train, label=y_train)

# enable auto logging
mlflow.lightgbm.autolog()

with mlflow.start_run():

    # train model
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "learning_rate": learning_rate,
        "metric": "multi_logloss",
        "colsample_bytree": colsample_bytree,
        "subsample": subsample,
        "seed": 42,
    }
    model = lgb.train(
        params, train_set, num_boost_round=10, valid_sets=[train_set], valid_names=["train"]
    )

    # evaluate model
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    loss = log_loss(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    # log metrics
    mlflow.log_metrics({"log_loss": loss, "accuracy": acc})
    mlflow.lightgbm.log_model(model,"model")

/opt/conda/lib/python3.7/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505
/opt/conda/lib/python3.7/site-packages/mlflow/lightgbm.py:285: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_arg_names = inspect.getargspec(original)[0]  # pylint: disable=W1505


[1]	train's multi_logloss: 0.669599
[2]	train's multi_logloss: 0.460717
[3]	train's multi_logloss: 0.333111
[4]	train's multi_logloss: 0.258027
[5]	train's multi_logloss: 0.202322
[6]	train's multi_logloss: 0.1656
[7]	train's multi_logloss: 0.140064
[8]	train's multi_logloss: 0.120381
[9]	train's multi_logloss: 0.103389
[10]	train's multi_logloss: 0.0919157


/opt/conda/lib/python3.7/site-packages/mlflow/lightgbm.py:262: UserWarning: Logging to MLflow failed: No module named 'boto3'
  try_mlflow_log(mlflow.log_artifact, filepath)
/opt/conda/lib/python3.7/site-packages/mlflow/lightgbm.py:338: UserWarning: Logging to MLflow failed: No module named 'boto3'
  try_mlflow_log(mlflow.log_artifact, filepath)
/opt/conda/lib/python3.7/site-packages/mlflow/lightgbm.py:342: UserWarning: Logging to MLflow failed: No module named 'boto3'
  try_mlflow_log(log_model, model, artifact_path='model')


TypeError: log_model() missing 1 required positional argument: 'artifact_path'